In [1]:
import streamlit as st
import cv2
import numpy as np
from PIL import Image
import pytesseract
import re
import spacy
from googletrans import Translator
import pandas as pd
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.units import inch
import io
import base64
from datetime import datetime
import json
import requests
from typing import Dict, List, Tuple, Optional

# Configuration
st.set_page_config(
    page_title="Medical Prescription Reader & Simplifier",
    page_icon="💊",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for better UI
st.markdown("""
<style>
    .main-header {
        text-align: center;
        color: #2E86AB;
        font-size: 2.5rem;
        margin-bottom: 2rem;
    }
    .section-header {
        color: #F24236;
        font-size: 1.5rem;
        margin-top: 2rem;
        margin-bottom: 1rem;
    }
    .info-box {
        background-color: #E8F4FD;
        padding: 1rem;
        border-radius: 0.5rem;
        border-left: 4px solid #2E86AB;
        margin: 1rem 0;
    }
    .warning-box {
        background-color: #FFF3CD;
        padding: 1rem;
        border-radius: 0.5rem;
        border-left: 4px solid #F24236;
        margin: 1rem 0;
    }
    .success-box {
        background-color: #D4EDDA;
        padding: 1rem;
        border-radius: 0.5rem;
        border-left: 4px solid #28A745;
        margin: 1rem 0;
    }
</style>
""", unsafe_allow_html=True)

class PrescriptionProcessor:
    def __init__(self):
        self.translator = Translator()
        self.medical_abbreviations = {
            # Frequency abbreviations
            'OD': 'once daily',
            'BD': 'twice daily',
            'BID': 'twice daily',
            'TID': 'three times daily',
            'QID': 'four times daily',
            'PRN': 'as needed',
            'SOS': 'if required',
            'QH': 'every hour',
            'Q2H': 'every 2 hours',
            'Q4H': 'every 4 hours',
            'Q6H': 'every 6 hours',
            'Q8H': 'every 8 hours',
            'Q12H': 'every 12 hours',
            'HS': 'at bedtime',
            'AC': 'before meals',
            'PC': 'after meals',
            'STAT': 'immediately',

            # Dosage forms
            'TAB': 'tablet',
            'TABS': 'tablets',
            'CAP': 'capsule',
            'CAPS': 'capsules',
            'SYP': 'syrup',
            'SYRUP': 'syrup',
            'INJ': 'injection',
            'DROPS': 'drops',
            'OINT': 'ointment',
            'CREAM': 'cream',
            'GEL': 'gel',
            'LOTION': 'lotion',
            'POWDER': 'powder',
            'SACHETS': 'sachets',

            # Routes
            'PO': 'by mouth',
            'IV': 'intravenous',
            'IM': 'intramuscular',
            'SC': 'subcutaneous',
            'TOP': 'topical',
            'EYE': 'eye',
            'EAR': 'ear',
            'NASAL': 'nasal',

            # Units
            'MG': 'milligrams',
            'G': 'grams',
            'ML': 'milliliters',
            'L': 'liters',
            'IU': 'international units',
            'MCG': 'micrograms',
            'TSP': 'teaspoon',
            'TBSP': 'tablespoon',
        }

        self.languages = {
            'English': 'en',
            'Hindi': 'hi',
            'Marathi': 'mr',
            'Tamil': 'ta',
            'Telugu': 'te',
            'Bengali': 'bn',
            'Gujarati': 'gu',
            'Kannada': 'kn',
            'Malayalam': 'ml',
            'Punjabi': 'pa',
            'Urdu': 'ur'
        }

        self.common_medicines = {
            'paracetamol': 'fever and pain reliever',
            'aspirin': 'pain reliever and blood thinner',
            'ibuprofen': 'pain and inflammation reliever',
            'amoxicillin': 'antibiotic',
            'azithromycin': 'antibiotic',
            'metformin': 'diabetes medication',
            'amlodipine': 'blood pressure medication',
            'atorvastatin': 'cholesterol medication',
            'omeprazole': 'acid reflux medication',
            'cetirizine': 'allergy medication',
            'pantoprazole': 'acid reflux medication',
            'losartan': 'blood pressure medication',
            'clopidogrel': 'blood thinner',
            'levothyroxine': 'thyroid medication',
            'insulin': 'diabetes medication'
        }

    def preprocess_image(self, image):
        """Preprocess image for better OCR results"""
        # Convert PIL image to numpy array
        img_array = np.array(image)

        # Convert to grayscale
        gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)

        # Apply denoising
        denoised = cv2.fastNlMeansDenoising(gray)

        # Apply adaptive thresholding
        thresh = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                     cv2.THRESH_BINARY, 11, 2)

        # Apply morphological operations to clean up the image
        kernel = np.ones((1, 1), np.uint8)
        cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

        return cleaned

    def extract_text_from_image(self, image):
        """Extract text from image using OCR"""
        try:
            # Preprocess the image
            processed_img = self.preprocess_image(image)

            # Configure Tesseract for better medical text recognition
            custom_config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz.,()-:/'

            # Extract text
            text = pytesseract.image_to_string(processed_img, config=custom_config)

            return text.strip()
        except Exception as e:
            st.error(f"Error extracting text: {str(e)}")
            return ""

    def clean_text(self, text):
        """Clean and preprocess extracted text"""
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text)

        # Remove special characters but keep medical ones
        text = re.sub(r'[^\w\s\-\(\)\.\,\:\;\/]', '', text)

        # Convert to uppercase for consistency
        text = text.upper()

        return text.strip()

    def extract_medicine_info(self, text):
        """Extract medicine information using NLP and regex"""
        medicines = []
        lines = text.split('\n')

        for line in lines:
            line = line.strip()
            if not line:
                continue

            # Extract medicine name (usually starts the line)
            medicine_match = re.search(r'^([A-Z][A-Z\s]+?)(?:\s+(?:TAB|CAP|SYP|INJ|DROPS)|\s+\d)', line)
            if medicine_match:
                medicine_name = medicine_match.group(1).strip()

                # Extract dosage information
                dosage_pattern = r'(\d+(?:\.\d+)?)\s*(?:MG|G|ML|MCG|IU)?'
                dosage_match = re.search(dosage_pattern, line)
                dosage = dosage_match.group(0) if dosage_match else "Not specified"

                # Extract frequency patterns
                frequency_patterns = [
                    r'(\d+\-\d+\-\d+)',  # Pattern like 1-0-1
                    r'(\d+)\s*(?:X|TIMES?)\s*(?:DAILY|DAY|A\s*DAY)',  # Pattern like 2 times daily
                    r'(OD|BD|BID|TID|QID|PRN|SOS|HS|AC|PC|STAT)',  # Medical abbreviations
                    r'(\d+)\s*(?:HOURLY|HOURS?)',  # Hourly patterns
                ]

                frequency = "Not specified"
                for pattern in frequency_patterns:
                    freq_match = re.search(pattern, line)
                    if freq_match:
                        frequency = freq_match.group(1)
                        break

                # Extract duration
                duration_pattern = r'(?:FOR|X)\s*(\d+)\s*(?:DAYS?|WEEKS?|MONTHS?)'
                duration_match = re.search(duration_pattern, line)
                duration = duration_match.group(0) if duration_match else "Not specified"

                # Extract timing (before/after meals)
                timing_pattern = r'(BEFORE|AFTER|WITH)\s*(?:MEALS?|FOOD|BREAKFAST|LUNCH|DINNER)'
                timing_match = re.search(timing_pattern, line)
                timing = timing_match.group(0) if timing_match else "Not specified"

                medicines.append({
                    'name': medicine_name,
                    'dosage': dosage,
                    'frequency': frequency,
                    'duration': duration,
                    'timing': timing,
                    'original_text': line
                })

        return medicines

    def convert_to_plain_language(self, medicines):
        """Convert medical shorthand to plain language"""
        simplified_instructions = []

        for med in medicines:
            instruction = f"**{med['name']}**\n\n"

            # Convert frequency to plain language
            freq = med['frequency']
            if re.match(r'\d+\-\d+\-\d+', freq):
                parts = freq.split('-')
                morning, afternoon, night = parts[0], parts[1], parts[2]
                freq_text = []
                if morning != '0':
                    freq_text.append(f"{morning} in the morning")
                if afternoon != '0':
                    freq_text.append(f"{afternoon} in the afternoon")
                if night != '0':
                    freq_text.append(f"{night} at night")
                freq_plain = " and ".join(freq_text)
            else:
                freq_plain = self.medical_abbreviations.get(freq, freq)

            # Build instruction
            if med['dosage'] != "Not specified":
                instruction += f"• Take {med['dosage']} "
            else:
                instruction += f"• Take as prescribed "

            instruction += f"{freq_plain}"

            if med['timing'] != "Not specified":
                instruction += f" {med['timing'].lower()}"

            if med['duration'] != "Not specified":
                instruction += f" {med['duration'].lower()}"

            # Add medicine information if available
            med_name_lower = med['name'].lower()
            for common_med, info in self.common_medicines.items():
                if common_med in med_name_lower:
                    instruction += f"\n• This medicine is used for: {info}"
                    break

            instruction += "\n"
            simplified_instructions.append(instruction)

        return simplified_instructions

    def translate_text(self, text, target_language):
        """Translate text to target language"""
        try:
            if target_language == 'en':
                return text

            translated = self.translator.translate(text, dest=target_language)
            return translated.text
        except Exception as e:
            st.error(f"Translation error: {str(e)}")
            return text

    def generate_pdf_report(self, medicines, instructions, patient_name="Patient"):
        """Generate PDF report of simplified prescription"""
        buffer = io.BytesIO()
        doc = SimpleDocTemplate(buffer, pagesize=letter)
        styles = getSampleStyleSheet()
        story = []

        # Title
        title_style = ParagraphStyle(
            'CustomTitle',
            parent=styles['Heading1'],
            fontSize=24,
            spaceAfter=30,
            textColor='blue'
        )
        story.append(Paragraph("Simplified Prescription Report", title_style))
        story.append(Spacer(1, 20))

        # Patient info
        story.append(Paragraph(f"<b>Patient:</b> {patient_name}", styles['Normal']))
        story.append(Paragraph(f"<b>Date:</b> {datetime.now().strftime('%B %d, %Y')}", styles['Normal']))
        story.append(Spacer(1, 20))

        # Instructions
        story.append(Paragraph("<b>Medication Instructions:</b>", styles['Heading2']))
        story.append(Spacer(1, 12))

        for instruction in instructions:
            story.append(Paragraph(instruction.replace('**', '<b>').replace('**', '</b>'), styles['Normal']))
            story.append(Spacer(1, 12))

        # Important notes
        story.append(Paragraph("<b>Important Notes:</b>", styles['Heading2']))
        notes = [
            "• Always complete the full course of antibiotics even if you feel better",
            "• Take medications at the same time each day",
            "• Do not skip doses",
            "• Contact your doctor if you experience any side effects",
            "• Store medications in a cool, dry place away from children"
        ]

        for note in notes:
            story.append(Paragraph(note, styles['Normal']))
            story.append(Spacer(1, 6))

        doc.build(story)
        buffer.seek(0)
        return buffer

def main():
    st.markdown('<h1 class="main-header">💊 Medical Prescription Reader & Simplifier</h1>', unsafe_allow_html=True)

    # Initialize processor
    processor = PrescriptionProcessor()

    # Sidebar
    st.sidebar.header("Settings")
    selected_language = st.sidebar.selectbox(
        "Select Output Language",
        list(processor.languages.keys()),
        index=0
    )

    patient_name = st.sidebar.text_input("Patient Name (Optional)", "")

    # Info box
    st.markdown("""
    <div class="info-box">
        <h3>🎯 How it works:</h3>
        <p>1. Upload your prescription image</p>
        <p>2. Our AI will read and extract medicine information</p>
        <p>3. Get simplified, easy-to-understand instructions</p>
        <p>4. Download or share the patient-friendly summary</p>
    </div>
    """, unsafe_allow_html=True)

    # File upload
    st.markdown('<h2 class="section-header">📤 Upload Prescription</h2>', unsafe_allow_html=True)

    upload_method = st.radio(
        "Choose upload method:",
        ["Upload Image File", "Take Photo with Camera"]
    )

    uploaded_file = None

    if upload_method == "Upload Image File":
        uploaded_file = st.file_uploader(
            "Choose a prescription image",
            type=['png', 'jpg', 'jpeg', 'bmp', 'tiff'],
            help="Upload a clear image of your prescription"
        )
    else:
        camera_input = st.camera_input("Take a photo of your prescription")
        if camera_input:
            uploaded_file = camera_input

    if uploaded_file is not None:
        # Display uploaded image
        image = Image.open(uploaded_file)
        col1, col2 = st.columns([1, 2])

        with col1:
            st.image(image, caption="Uploaded Prescription", use_column_width=True)

        with col2:
            st.markdown('<h2 class="section-header">🔍 Processing Results</h2>', unsafe_allow_html=True)

            with st.spinner("Processing prescription..."):
                # Extract text
                extracted_text = processor.extract_text_from_image(image)

                if extracted_text:
                    st.success("✅ Text extracted successfully!")

                    # Show extracted text in expandable section
                    with st.expander("View Extracted Text"):
                        st.text_area("Raw OCR Output", extracted_text, height=150)

                    # Clean and process text
                    cleaned_text = processor.clean_text(extracted_text)

                    # Extract medicine information
                    medicines = processor.extract_medicine_info(cleaned_text)

                    if medicines:
                        st.markdown('<h2 class="section-header">💊 Extracted Medicines</h2>', unsafe_allow_html=True)

                        # Display medicines in a table
                        df = pd.DataFrame(medicines)
                        st.dataframe(df[['name', 'dosage', 'frequency', 'duration', 'timing']], use_container_width=True)

                        # Convert to plain language
                        instructions = processor.convert_to_plain_language(medicines)

                        st.markdown('<h2 class="section-header">📋 Simplified Instructions</h2>', unsafe_allow_html=True)

                        # Translate if needed
                        if selected_language != 'English':
                            translated_instructions = []
                            for instruction in instructions:
                                translated = processor.translate_text(
                                    instruction,
                                    processor.languages[selected_language]
                                )
                                translated_instructions.append(translated)
                            instructions = translated_instructions

                        # Display instructions
                        for i, instruction in enumerate(instructions, 1):
                            st.markdown(f"""
                            <div class="success-box">
                                <h4>Medicine {i}:</h4>
                                {instruction.replace('**', '<b>').replace('**', '</b>').replace('•', '→')}
                            </div>
                            """, unsafe_allow_html=True)

                        # Download options
                        st.markdown('<h2 class="section-header">📥 Download & Share</h2>', unsafe_allow_html=True)

                        col1, col2, col3 = st.columns(3)

                        with col1:
                            # Generate PDF
                            pdf_buffer = processor.generate_pdf_report(
                                medicines,
                                instructions,
                                patient_name or "Patient"
                            )

                            st.download_button(
                                label="📄 Download PDF Report",
                                data=pdf_buffer,
                                file_name=f"prescription_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf",
                                mime="application/pdf"
                            )

                        with col2:
                            # Generate text summary
                            text_summary = f"Prescription Summary for {patient_name or 'Patient'}\n"
                            text_summary += f"Date: {datetime.now().strftime('%B %d, %Y')}\n\n"
                            text_summary += "MEDICATION INSTRUCTIONS:\n"
                            text_summary += "=" * 50 + "\n\n"

                            for instruction in instructions:
                                text_summary += instruction.replace('**', '').replace('•', '→') + "\n"

                            text_summary += "\nIMPORTANT REMINDERS:\n"
                            text_summary += "- Complete full course of antibiotics\n"
                            text_summary += "- Take medications at same time daily\n"
                            text_summary += "- Contact doctor for side effects\n"

                            st.download_button(
                                label="📝 Download Text Summary",
                                data=text_summary,
                                file_name=f"prescription_summary_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt",
                                mime="text/plain"
                            )

                        with col3:
                            # WhatsApp sharing
                            whatsapp_text = "Prescription%20Summary%3A%0A"
                            for instruction in instructions:
                                clean_instruction = instruction.replace('**', '').replace('•', '→').replace('\n', '%0A')
                                whatsapp_text += clean_instruction + "%0A%0A"

                            whatsapp_url = f"https://wa.me/?text={whatsapp_text}"
                            st.markdown(
                                f'<a href="{whatsapp_url}" target="_blank" style="text-decoration: none;">'
                                f'<button style="background-color: #25D366; color: white; border: none; '
                                f'padding: 10px 20px; border-radius: 5px; cursor: pointer;">'
                                f'📱 Share on WhatsApp</button></a>',
                                unsafe_allow_html=True
                            )

                    else:
                        st.markdown("""
                        <div class="warning-box">
                            <h4>⚠️ No medicines detected</h4>
                            <p>The system couldn't detect any medicine information in the prescription.
                            This could be due to:</p>
                            <ul>
                                <li>Poor image quality</li>
                                <li>Handwriting is too unclear</li>
                                <li>Image is rotated or skewed</li>
                                <li>Non-standard prescription format</li>
                            </ul>
                            <p>Please try uploading a clearer image or contact support.</p>
                        </div>
                        """, unsafe_allow_html=True)

                else:
                    st.error("❌ Could not extract text from the image. Please try with a clearer image.")

    else:
        st.info("👆 Please upload a prescription image to get started")

    # Footer
    st.markdown("---")
    st.markdown("""
    <div style="text-align: center; color: gray; font-size: 0.9em;">
        <p>🏥 Medical Prescription Reader & Simplifier</p>
        <p>⚠️ This tool is for assistance only. Always consult with your healthcare provider for medical advice.</p>
    </div>
    """, unsafe_allow_html=True)

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'streamlit'

In [9]:

!pip install streamlit pyngrok --quiet


In [2]:
!pip install streamlit opencv-python pillow pytesseract spacy googletrans==4.0.0-rc1 pandas reportlab

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.8 MB/s eta 0:00:00
  Created wheel for g

In [4]:

import cv2
import numpy as np
from PIL import Image
import pytesseract
import re
import spacy
from googletrans import Translator
import pandas as pd
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.units import inch
import io
import base64
from datetime import datetime
import json
import requests
from typing import Dict, List, Tuple, Optional

In [7]:
import streamlit as st
from googletrans import Translator

st.set_page_config(
    page_title="Medical Prescription Reader & Simplifier",
    page_icon="💊",
    layout="wide",
    initial_sidebar_state="expanded"
)

st.set_page_config(
    page_title="Medical Prescription Reader & Simplifier",
    page_icon="💊",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for better UI
st.markdown("""
<style>
    .main-header {
        text-align: center;
        color: #2E86AB;
        font-size: 2.5rem;
        margin-bottom: 2rem;
    }
    .section-header {
        color: #F24236;
        font-size: 1.5rem;
        margin-top: 2rem;
        margin-bottom: 1rem;
    }
    .info-box {
        background-color: #E8F4FD;
        padding: 1rem;
        border-radius: 0.5rem;
        border-left: 4px solid #2E86AB;
        margin: 1rem 0;
    }
    .warning-box {
        background-color: #FFF3CD;
        padding: 1rem;
        border-radius: 0.5rem;
        border-left: 4px solid #F24236;
        margin: 1rem 0;
    }
    .success-box {
        background-color: #D4EDDA;
        padding: 1rem;
        border-radius: 0.5rem;
        border-left: 4px solid #28A745;
        margin: 1rem 0;
    }
</style>
""", unsafe_allow_html=True)

class PrescriptionProcessor:
    def __init__(self):
        self.translator = Translator()
        self.medical_abbreviations = {
            # Frequency abbreviations
            'OD': 'once daily',
            'BD': 'twice daily',
            'BID': 'twice daily',
            'TID': 'three times daily',
            'QID': 'four times daily',
            'PRN': 'as needed',
            'SOS': 'if required',
            'QH': 'every hour',
            'Q2H': 'every 2 hours',
            'Q4H': 'every 4 hours',
            'Q6H': 'every 6 hours',
            'Q8H': 'every 8 hours',
            'Q12H': 'every 12 hours',
            'HS': 'at bedtime',
            'AC': 'before meals',
            'PC': 'after meals',
            'STAT': 'immediately',

            # Dosage forms
            'TAB': 'tablet',
            'TABS': 'tablets',
            'CAP': 'capsule',
            'CAPS': 'capsules',
            'SYP': 'syrup',
            'SYRUP': 'syrup',
            'INJ': 'injection',
            'DROPS': 'drops',
            'OINT': 'ointment',
            'CREAM': 'cream',
            'GEL': 'gel',
            'LOTION': 'lotion',
            'POWDER': 'powder',
            'SACHETS': 'sachets',

            # Routes
            'PO': 'by mouth',
            'IV': 'intravenous',
            'IM': 'intramuscular',
            'SC': 'subcutaneous',
            'TOP': 'topical',
            'EYE': 'eye',
            'EAR': 'ear',
            'NASAL': 'nasal',

            # Units
            'MG': 'milligrams',
            'G': 'grams',
            'ML': 'milliliters',
            'L': 'liters',
            'IU': 'international units',
            'MCG': 'micrograms',
            'TSP': 'teaspoon',
            'TBSP': 'tablespoon',
        }

        self.languages = {
            'English': 'en',
            'Hindi': 'hi',
            'Marathi': 'mr',
            'Tamil': 'ta',
            'Telugu': 'te',
            'Bengali': 'bn',
            'Gujarati': 'gu',
            'Kannada': 'kn',
            'Malayalam': 'ml',
            'Punjabi': 'pa',
            'Urdu': 'ur'
        }

        self.common_medicines = {
            'paracetamol': 'fever and pain reliever',
            'aspirin': 'pain reliever and blood thinner',
            'ibuprofen': 'pain and inflammation reliever',
            'amoxicillin': 'antibiotic',
            'azithromycin': 'antibiotic',
            'metformin': 'diabetes medication',
            'amlodipine': 'blood pressure medication',
            'atorvastatin': 'cholesterol medication',
            'omeprazole': 'acid reflux medication',
            'cetirizine': 'allergy medication',
            'pantoprazole': 'acid reflux medication',
            'losartan': 'blood pressure medication',
            'clopidogrel': 'blood thinner',
            'levothyroxine': 'thyroid medication',
            'insulin': 'diabetes medication'
        }

2025-07-13 12:53:42.424 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-13 12:53:42.426 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-13 12:53:42.427 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-13 12:53:42.513 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-07-13 12:53:42.514 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-13 12:53:42.515 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [8]:
app_code = """
import streamlit as st
st.set_page_config(page_title='Hello from Colab', page_icon='👋')

st.title("✅ Streamlit on Colab")
st.write("This is a working demo of Streamlit running on Google Colab using pyngrok.")
"""
with open("app.py", "w") as f:
    f.write(app_code)

In [10]:
from pyngrok import ngrok

# Kill any existing tunnels (if restarting)
ngrok.kill()

# Open a tunnel on port 8501 (Streamlit default)
public_url = ngrok.connect(8501)
print(f"🔗 Your Streamlit app is available at: {public_url}")


ERROR:pyngrok.process.ngrok:t=2025-07-13T12:56:01+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [13]:


def preprocess_image(self, image):
        """Preprocess image for better OCR results"""
        # Convert PIL image to numpy array
        img_array = np.array(image)

        # Convert to grayscale
        gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)

        # Apply denoising
        denoised = cv2.fastNlMeansDenoising(gray)

        # Apply adaptive thresholding
        thresh = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                     cv2.THRESH_BINARY, 11, 2)

        # Apply morphological operations to clean up the image
        kernel = np.ones((1, 1), np.uint8)
        cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

        return cleaned

def extract_text_from_image(self, image):
        """Extract text from image using OCR"""
        try:
            # Preprocess the image
            processed_img = self.preprocess_image(image)

            # Configure Tesseract for better medical text recognition
            custom_config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz.,()-:/'

            # Extract text
            text = pytesseract.image_to_string(processed_img, config=custom_config)

            return text.strip()
        except Exception as e:
            st.error(f"Error extracting text: {str(e)}")
            return ""

def clean_text(self, text):
        """Clean and preprocess extracted text"""
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text)

        # Remove special characters but keep medical ones
        text = re.sub(r'[^\w\s\-\(\)\.\,\:\;\/]', '', text)

        # Convert to uppercase for consistency
        text = text.upper()

        return text.strip()

def extract_medicine_info(self, text):
        """Extract medicine information using NLP and regex"""
        medicines = []
        lines = text.split('\n')

        for line in lines:
            line = line.strip()
            if not line:
                continue

            # Extract medicine name (usually starts the line)
            medicine_match = re.search(r'^([A-Z][A-Z\s]+?)(?:\s+(?:TAB|CAP|SYP|INJ|DROPS)|\s+\d)', line)
            if medicine_match:
                medicine_name = medicine_match.group(1).strip()

                # Extract dosage information
                dosage_pattern = r'(\d+(?:\.\d+)?)\s*(?:MG|G|ML|MCG|IU)?'
                dosage_match = re.search(dosage_pattern, line)
                dosage = dosage_match.group(0) if dosage_match else "Not specified"

                # Extract frequency patterns
                frequency_patterns = [
                    r'(\d+\-\d+\-\d+)',  # Pattern like 1-0-1
                    r'(\d+)\s*(?:X|TIMES?)\s*(?:DAILY|DAY|A\s*DAY)',  # Pattern like 2 times daily
                    r'(OD|BD|BID|TID|QID|PRN|SOS|HS|AC|PC|STAT)',  # Medical abbreviations
                    r'(\d+)\s*(?:HOURLY|HOURS?)',  # Hourly patterns
                ]

                frequency = "Not specified"
                for pattern in frequency_patterns:
                    freq_match = re.search(pattern, line)
                    if freq_match:
                        frequency = freq_match.group(1)
                        break

                # Extract duration
                duration_pattern = r'(?:FOR|X)\s*(\d+)\s*(?:DAYS?|WEEKS?|MONTHS?)'
                duration_match = re.search(duration_pattern, line)
                duration = duration_match.group(0) if duration_match else "Not specified"

                # Extract timing (before/after meals)
                timing_pattern = r'(BEFORE|AFTER|WITH)\s*(?:MEALS?|FOOD|BREAKFAST|LUNCH|DINNER)'
                timing_match = re.search(timing_pattern, line)
                timing = timing_match.group(0) if timing_match else "Not specified"

                medicines.append({
                    'name': medicine_name,
                    'dosage': dosage,
                    'frequency': frequency,
                    'duration': duration,
                    'timing': timing,
                    'original_text': line
                })

        return medicines

def convert_to_plain_language(self, medicines):
        """Convert medical shorthand to plain language"""
        simplified_instructions = []

        for med in medicines:
            instruction = f"**{med['name']}**\n\n"

            # Convert frequency to plain language
            freq = med['frequency']
            if re.match(r'\d+\-\d+\-\d+', freq):
                parts = freq.split('-')
                morning, afternoon, night = parts[0], parts[1], parts[2]
                freq_text = []
                if morning != '0':
                    freq_text.append(f"{morning} in the morning")
                if afternoon != '0':
                    freq_text.append(f"{afternoon} in the afternoon")
                if night != '0':
                    freq_text.append(f"{night} at night")
                freq_plain = " and ".join(freq_text)
            else:
                freq_plain = self.medical_abbreviations.get(freq, freq)

            # Build instruction
            if med['dosage'] != "Not specified":
                instruction += f"• Take {med['dosage']} "
            else:
                instruction += f"• Take as prescribed "

            instruction += f"{freq_plain}"

            if med['timing'] != "Not specified":
                instruction += f" {med['timing'].lower()}"

            if med['duration'] != "Not specified":
                instruction += f" {med['duration'].lower()}"

            # Add medicine information if available
            med_name_lower = med['name'].lower()
            for common_med, info in self.common_medicines.items():
                if common_med in med_name_lower:
                    instruction += f"\n• This medicine is used for: {info}"
                    break

            instruction += "\n"
            simplified_instructions.append(instruction)

        return simplified_instructions

def translate_text(self, text, target_language):
        """Translate text to target language"""
        try:
            if target_language == 'en':
                return text

            translated = self.translator.translate(text, dest=target_language)
            return translated.text
        except Exception as e:
            st.error(f"Translation error: {str(e)}")
            return text

def generate_pdf_report(self, medicines, instructions, patient_name="Patient"):
        """Generate PDF report of simplified prescription"""
        buffer = io.BytesIO()
        doc = SimpleDocTemplate(buffer, pagesize=letter)
        styles = getSampleStyleSheet()
        story = []

        # Title
        title_style = ParagraphStyle(
            'CustomTitle',
            parent=styles['Heading1'],
            fontSize=24,
            spaceAfter=30,
            textColor='blue'
        )
        story.append(Paragraph("Simplified Prescription Report", title_style))
        story.append(Spacer(1, 20))

        # Patient info
        story.append(Paragraph(f"<b>Patient:</b> {patient_name}", styles['Normal']))
        story.append(Paragraph(f"<b>Date:</b> {datetime.now().strftime('%B %d, %Y')}", styles['Normal']))
        story.append(Spacer(1, 20))

        # Instructions
        story.append(Paragraph("<b>Medication Instructions:</b>", styles['Heading2']))
        story.append(Spacer(1, 12))

        for instruction in instructions:
            story.append(Paragraph(instruction.replace('**', '<b>').replace('**', '</b>'), styles['Normal']))
            story.append(Spacer(1, 12))

        # Important notes
        story.append(Paragraph("<b>Important Notes:</b>", styles['Heading2']))
        notes = [
            "• Always complete the full course of antibiotics even if you feel better",
            "• Take medications at the same time each day",
            "• Do not skip doses",
            "• Contact your doctor if you experience any side effects",
            "• Store medications in a cool, dry place away from children"
        ]

        for note in notes:
            story.append(Paragraph(note, styles['Normal']))
            story.append(Spacer(1, 6))

        doc.build(story)
        buffer.seek(0)
        return buffer

def main():
    st.markdown('<h1 class="main-header">💊 Medical Prescription Reader & Simplifier</h1>', unsafe_allow_html=True)

    # Initialize processor
    processor = PrescriptionProcessor()

    # Sidebar
    st.sidebar.header("Settings")
    selected_language = st.sidebar.selectbox(
        "Select Output Language",
        list(processor.languages.keys()),
        index=0
    )

    patient_name = st.sidebar.text_input("Patient Name (Optional)", "")

    # Info box
    st.markdown("""
    <div class="info-box">
        <h3>🎯 How it works:</h3>
        <p>1. Upload your prescription image</p>
        <p>2. Our AI will read and extract medicine information</p>
        <p>3. Get simplified, easy-to-understand instructions</p>
        <p>4. Download or share the patient-friendly summary</p>
    </div>
    """, unsafe_allow_html=True)

    # File upload
    st.markdown('<h2 class="section-header">📤 Upload Prescription</h2>', unsafe_allow_html=True)

    upload_method = st.radio(
        "Choose upload method:",
        ["Upload Image File", "Take Photo with Camera"]
    )

    uploaded_file = None

    if upload_method == "Upload Image File":
        uploaded_file = st.file_uploader(
            "Choose a prescription image",
            type=['png', 'jpg', 'jpeg', 'bmp', 'tiff'],
            help="Upload a clear image of your prescription"
        )
    else:
        camera_input = st.camera_input("Take a photo of your prescription")
        if camera_input:
            uploaded_file = camera_input

    if uploaded_file is not None:
        # Display uploaded image
        image = Image.open(uploaded_file)
        col1, col2 = st.columns([1, 2])

        with col1:
            st.image(image, caption="Uploaded Prescription", use_column_width=True)

        with col2:
            st.markdown('<h2 class="section-header">🔍 Processing Results</h2>', unsafe_allow_html=True)

            with st.spinner("Processing prescription..."):
                # Extract text
                extracted_text = processor.extract_text_from_image(image)

                if extracted_text:
                    st.success("✅ Text extracted successfully!")

                    # Show extracted text in expandable section
                    with st.expander("View Extracted Text"):
                        st.text_area("Raw OCR Output", extracted_text, height=150)

                    # Clean and process text
                    cleaned_text = processor.clean_text(extracted_text)

                    # Extract medicine information
                    medicines = processor.extract_medicine_info(cleaned_text)

                    if medicines:
                        st.markdown('<h2 class="section-header">💊 Extracted Medicines</h2>', unsafe_allow_html=True)

                        # Display medicines in a table
                        df = pd.DataFrame(medicines)
                        st.dataframe(df[['name', 'dosage', 'frequency', 'duration', 'timing']], use_container_width=True)

                        # Convert to plain language
                        instructions = processor.convert_to_plain_language(medicines)

                        st.markdown('<h2 class="section-header">📋 Simplified Instructions</h2>', unsafe_allow_html=True)

                        # Translate if needed
                        if selected_language != 'English':
                            translated_instructions = []
                            for instruction in instructions:
                                translated = processor.translate_text(
                                    instruction,
                                    processor.languages[selected_language]
                                )
                                translated_instructions.append(translated)
                            instructions = translated_instructions

                        # Display instructions
                        for i, instruction in enumerate(instructions, 1):
                            st.markdown(f"""
                            <div class="success-box">
                                <h4>Medicine {i}:</h4>
                                {instruction.replace('**', '<b>').replace('**', '</b>').replace('•', '→')}
                            </div>
                            """, unsafe_allow_html=True)

                        # Download options
                        st.markdown('<h2 class="section-header">📥 Download & Share</h2>', unsafe_allow_html=True)

                        col1, col2, col3 = st.columns(3)

                        with col1:
                            # Generate PDF
                            pdf_buffer = processor.generate_pdf_report(
                                medicines,
                                instructions,
                                patient_name or "Patient"
                            )

                            st.download_button(
                                label="📄 Download PDF Report",
                                data=pdf_buffer,
                                file_name=f"prescription_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf",
                                mime="application/pdf"
                            )

                        with col2:
                            # Generate text summary
                            text_summary = f"Prescription Summary for {patient_name or 'Patient'}\n"
                            text_summary += f"Date: {datetime.now().strftime('%B %d, %Y')}\n\n"
                            text_summary += "MEDICATION INSTRUCTIONS:\n"
                            text_summary += "=" * 50 + "\n\n"

                            for instruction in instructions:
                                text_summary += instruction.replace('**', '').replace('•', '→') + "\n"

                            text_summary += "\nIMPORTANT REMINDERS:\n"
                            text_summary += "- Complete full course of antibiotics\n"
                            text_summary += "- Take medications at same time daily\n"
                            text_summary += "- Contact doctor for side effects\n"

                            st.download_button(
                                label="📝 Download Text Summary",
                                data=text_summary,
                                file_name=f"prescription_summary_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt",
                                mime="text/plain"
                            )

                        with col3:
                            # WhatsApp sharing
                            whatsapp_text = "Prescription%20Summary%3A%0A"
                            for instruction in instructions:
                                clean_instruction = instruction.replace('**', '').replace('•', '→').replace('\n', '%0A')
                                whatsapp_text += clean_instruction + "%0A%0A"

                            whatsapp_url = f"https://wa.me/?text={whatsapp_text}"
                            st.markdown(
                                f'<a href="{whatsapp_url}" target="_blank" style="text-decoration: none;">'
                                f'<button style="background-color: #25D366; color: white; border: none; '
                                f'padding: 10px 20px; border-radius: 5px; cursor: pointer;">'
                                f'📱 Share on WhatsApp</button></a>',
                                unsafe_allow_html=True
                            )

                    else:
                        st.markdown("""
                        <div class="warning-box">
                            <h4>⚠️ No medicines detected</h4>
                            <p>The system couldn't detect any medicine information in the prescription.
                            This could be due to:</p>
                            <ul>
                                <li>Poor image quality</li>
                                <li>Handwriting is too unclear</li>
                                <li>Image is rotated or skewed</li>
                                <li>Non-standard prescription format</li>
                            </ul>
                            <p>Please try uploading a clearer image or contact support.</p>
                        </div>
                        """, unsafe_allow_html=True)

                else:
                    st.error("❌ Could not extract text from the image. Please try with a clearer image.")

    else:
        st.info("👆 Please upload a prescription image to get started")

    # Footer
    st.markdown("---")
    st.markdown("""
    <div style="text-align: center; color: gray; font-size: 0.9em;">
        <p>🏥 Medical Prescription Reader & Simplifier</p>
        <p>⚠️ This tool is for assistance only. Always consult with your healthcare provider for medical advice.</p>
    </div>
    """, unsafe_allow_html=True)

if __name__ == "__main__":
    main()

2025-07-13 13:05:27.428 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-13 13:05:27.429 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-13 13:05:27.431 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-13 13:05:27.516 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-13 13:05:27.517 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-13 13:05:27.519 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-13 13:05:27.520 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-13 13:05:27.521 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar